In [16]:
# новый крутой вариант
import numpy as np
import matplotlib.pyplot as plt
import time
from pyDOE import lhs  # Библиотека для латинского гиперкуба

# Универсальная функция кэширования
def cache_function(func):
    cache = {}

    def cached_func(x):
        key = tuple(x) if isinstance(x, (list, np.ndarray)) else x
        
        # Если результат уже вычислен, возвращаем его
        if key in cache:
            return cache[key]
        
        # Иначе вычисляем результат и сохраняем его в кеш
        result = func(x)  # Вызываем переданную целевую функцию
        cache[key] = result
        return result
    
    return cached_func

# Пример целевой функции (Швефеля с 10 переменными)
def schwefel_function(x):
    return 418.9829 * len(x) - np.sum(x * np.sin(np.sqrt(np.abs(x))))  # Поддержка многомерности

# Кэшированная версия целевой функции
cached_objective_function = cache_function(schwefel_function)

# Инициализация популяции (выбор между случайной и латинским гиперкубом)
def initialize_population(pop_size, x_range, num_dimensions, method='random'):
    if method == 'random':
        # Случайное распределение для многомерной популяции
        return np.random.uniform(x_range[0], x_range[1], (pop_size, num_dimensions))
    elif method == 'lhs':  # Латинский гиперкуб
        lhs_samples = lhs(num_dimensions, samples=pop_size)  # num_dimensions измерений, pop_size выборок
        population = lhs_samples * (x_range[1] - x_range[0]) + x_range[0]  # Масштабируем выборки на нужный диапазон
        return population

# Селекция лучших индивидов с элитизмом (10% элиты)
def select_top_individuals_with_elitism(population, fitness_values, selection_percentage, elite_count=1):
    num_selected = int(len(population) * selection_percentage) - elite_count
    sorted_indices = np.argsort(fitness_values)
    elite_indices = sorted_indices[:elite_count]
    elite_population = population[elite_indices]
    elite_fitness = fitness_values[elite_indices]
    selected_indices = sorted_indices[elite_count:num_selected + elite_count]
    selected_population = population[selected_indices]
    selected_fitness = fitness_values[selected_indices]
    final_population = np.concatenate([elite_population, selected_population])
    final_fitness = np.concatenate([elite_fitness, selected_fitness])
    return final_population, final_fitness

# Скрещивание двух родителей
def crossover(parent1, parent2):
    return np.random.uniform(parent1, parent2)

# Создание потомков
def generate_offspring(selected_population, num_offspring):
    offspring = []
    for _ in range(num_offspring):
        parents = np.random.choice(len(selected_population), 2, replace=False)
        child = crossover(selected_population[parents[0]], selected_population[parents[1]])
        offspring.append(child)
    return np.array(offspring)

# Мутация потомков с новыми гиперпараметрами
def mutate(offspring, mutation_rate, mutation_strength, mutation_percent_genes, mutation_by_replacement, x_range):
    num_to_mutate = int(len(offspring) * mutation_rate)  # Сколько потомков подвергнутся мутации
    indices_to_mutate = np.random.choice(len(offspring), num_to_mutate, replace=False)

    for i in indices_to_mutate:  # Для каждого выбранного потомка
        num_genes_to_mutate = int(offspring.shape[1] * mutation_percent_genes)  # Сколько генов будет изменено
        genes_to_mutate = np.random.choice(offspring.shape[1], num_genes_to_mutate, replace=False)

        for gene_idx in genes_to_mutate:
            if mutation_by_replacement:
                # Полная замена гена на случайное значение
                offspring[i, gene_idx] = np.random.uniform(x_range[0], x_range[1])
            else:
                # Добавляем случайное смещение
                mutation = np.random.uniform(-mutation_strength, mutation_strength) * (x_range[1] - x_range[0])
                offspring[i, gene_idx] += mutation
                offspring[i, gene_idx] = np.clip(offspring[i, gene_idx], x_range[0], x_range[1])  # Ограничиваем диапазон

    return offspring

# Гиперпараметры
population_size = 500
x_range = (-500, 500)  # Диапазон для функции Швефеля
num_dimensions = 10  # Количество переменных (размерность)
selection_percentage = 0.5  # 50% популяции
offspring_ratio = 2.0  # Потомки в 2 раза больше родителей
mutation_rate = 0.5  # 30% потомков подвергаются мутации
mutation_strength = 0.3  # Сила мутации (30% диапазона)
mutation_percent_genes = 0.1  # 20% генов (параметров) будут изменены
mutation_by_replacement = True  # Полная замена генов при мутации
elite_fraction = 0.1  # 10% элитизма (1 особь)
num_iterations = 400  # Число итераций (гиперпараметр)
initialization_method = 'random'  # Способ инициализации: 'random' или 'lhs'
early_stopping_patience = 30  # Число итераций без улучшений до остановки
delta_threshold = 0.0001  # Минимальное изменение целевой функции для остановки

# Шаги
population = initialize_population(population_size, x_range, num_dimensions, method=initialization_method)
fitness_values = np.array([cached_objective_function(ind) for ind in population])

# Основной цикл
best_solution = None
best_fitness = float('inf')
no_improvement_count = 0  # Счётчик итераций без улучшений
delta_stability_count = 0  # Счетчик изменений по малому изменению

# Начало общего таймера для всех итераций
global_start_time = time.time()

for generation in range(num_iterations):
    start_time = time.time()  # Засекаем время начала итерации
    
    # Отбор лучших 50% индивидов с элитизмом
    selected_population, selected_fitness = select_top_individuals_with_elitism(population, fitness_values, selection_percentage, elite_count=int(elite_fraction * population_size))
    
    # Количество потомков
    num_offspring = int(len(selected_population) * offspring_ratio)
    offspring = generate_offspring(selected_population, num_offspring)

    # Мутация с новыми гиперпараметрами
    offspring = mutate(offspring, mutation_rate, mutation_strength, mutation_percent_genes, mutation_by_replacement, x_range)

    # Объединение родителей и потомков
    combined_population = np.concatenate([selected_population, offspring])
    combined_fitness = np.array([cached_objective_function(ind) for ind in combined_population])

    # Отбор лучших особей
    final_population, final_fitness = select_top_individuals_with_elitism(combined_population, combined_fitness, population_size / len(combined_population), elite_count=int(elite_fraction * population_size))

    # Обновляем популяцию
    population = final_population
    fitness_values = final_fitness

    # Находим лучшее решение
    current_best_fitness = np.min(final_fitness)
    current_best_solution = final_population[np.argmin(final_fitness)]
    
    fitness_delta = np.abs(current_best_fitness - best_fitness)
    
    # Условие обновления лучшего решения
    if current_best_fitness < best_fitness:
        best_fitness = current_best_fitness
        best_solution = current_best_solution
        no_improvement_count = 0  # Сброс счётчика при улучшении
        delta_stability_count = 0  # Сброс счетчика изменений
    else:
        no_improvement_count += 1  # Увеличиваем счётчик если нет улучшения

        # Проверка изменений целевой функции
        if fitness_delta < delta_threshold:
            delta_stability_count += 1
        else:
            delta_stability_count = 0  # Сброс если изменения есть

    # Время итерации и общее время с начала всего процесса
    iteration_time = time.time() - global_start_time

    # Вывод результатов итерации
    print(f"Итерация {generation+1}, значение цф = {best_fitness:.6f}, время = {iteration_time:.4f} секунд")

    # Early stopping: Если нет улучшений за последние 10 итераций, останавливаем алгоритм
    if no_improvement_count >= early_stopping_patience:
        print(f"Early stopping на {generation+1} итерации. Нет улучшений за {early_stopping_patience} итераций.")
        break



Итерация 1, значение цф = 2236.406213, время = 0.0850 секунд
Итерация 2, значение цф = 2236.406213, время = 0.1530 секунд
Итерация 3, значение цф = 2236.406213, время = 0.2020 секунд
Итерация 4, значение цф = 2236.406213, время = 0.2480 секунд
Итерация 5, значение цф = 2236.406213, время = 0.2880 секунд
Итерация 6, значение цф = 2236.406213, время = 0.3300 секунд
Итерация 7, значение цф = 2203.925936, время = 0.3640 секунд
Итерация 8, значение цф = 2111.766074, время = 0.3940 секунд
Итерация 9, значение цф = 2111.766074, время = 0.4260 секунд
Итерация 10, значение цф = 2111.766074, время = 0.4550 секунд
Итерация 11, значение цф = 2111.766074, время = 0.4840 секунд
Итерация 12, значение цф = 2111.766074, время = 0.5240 секунд
Итерация 13, значение цф = 2111.766074, время = 0.5780 секунд
Итерация 14, значение цф = 2111.766074, время = 0.6610 секунд
Итерация 15, значение цф = 2111.766074, время = 0.7270 секунд
Итерация 16, значение цф = 2111.766074, время = 0.7910 секунд
Итерация 17, знач

In [2]:
#вариант по 30*30 прогонов для исследования
import random
import numpy as np
import pandas as pd
import gc
import time
from pyDOE import lhs

# Универсальная функция кэширования с возможностью очистки кэша
class CacheFunction:
    def __init__(self, func):
        self.func = func
        self.cache = {}

    def __call__(self, x):
        key = tuple(x) if isinstance(x, (list, np.ndarray)) else x
        if key in self.cache:
            return self.cache[key]
        result = self.func(x)
        self.cache[key] = result
        return result

    def clear_cache(self):
        self.cache.clear()

# Пример целевой функции (Швефеля с 10 переменными)
def schwefel_function(x):
    return 418.9829 * len(x) - np.sum(x * np.sin(np.sqrt(np.abs(x))))

# Кэшированная версия целевой функции
cached_objective_function = CacheFunction(schwefel_function)

# Инициализация популяции (выбор между случайной и латинским гиперкубом)
def initialize_population(pop_size, x_range, num_dimensions, method='random'):
    if method == 'random':
        return np.random.uniform(x_range[0], x_range[1], (pop_size, num_dimensions))
    elif method == 'lhs':  # Латинский гиперкуб
        lhs_samples = lhs(num_dimensions, samples=pop_size)
        population = lhs_samples * (x_range[1] - x_range[0]) + x_range[0]
        return population

# Селекция лучших индивидов с элитизмом
def select_top_individuals_with_elitism(population, fitness_values, selection_percentage, elite_count=1):
    num_selected = int(len(population) * selection_percentage) - elite_count
    sorted_indices = np.argsort(fitness_values)
    elite_indices = sorted_indices[:elite_count]
    elite_population = population[elite_indices]
    elite_fitness = fitness_values[elite_indices]
    selected_indices = sorted_indices[elite_count:num_selected + elite_count]
    selected_population = population[selected_indices]
    selected_fitness = fitness_values[selected_indices]
    final_population = np.concatenate([elite_population, selected_population])
    final_fitness = np.concatenate([elite_fitness, selected_fitness])
    return final_population, final_fitness

# Скрещивание двух родителей
def crossover(parent1, parent2):
    return np.random.uniform(parent1, parent2)

# Создание потомков
def generate_offspring(selected_population, num_offspring):
    offspring = []
    for _ in range(num_offspring):
        parents = np.random.choice(len(selected_population), 2, replace=False)
        child = crossover(selected_population[parents[0]], selected_population[parents[1]])
        offspring.append(child)
    return np.array(offspring)

# Мутация потомков с новыми гиперпараметрами
def mutate(offspring, mutation_rate, mutation_strength, mutation_percent_genes, mutation_by_replacement, x_range):
    num_to_mutate = int(len(offspring) * mutation_rate)
    indices_to_mutate = np.random.choice(len(offspring), num_to_mutate, replace=False)
    for i in indices_to_mutate:
        num_genes_to_mutate = int(offspring.shape[1] * mutation_percent_genes)
        genes_to_mutate = np.random.choice(offspring.shape[1], num_genes_to_mutate, replace=False)
        for gene_idx in genes_to_mutate:
            if mutation_by_replacement:
                offspring[i, gene_idx] = np.random.uniform(x_range[0], x_range[1])
            else:
                mutation = np.random.uniform(-mutation_strength, mutation_strength) * (x_range[1] - x_range[0])
                offspring[i, gene_idx] += mutation
                offspring[i, gene_idx] = np.clip(offspring[i, gene_idx], x_range[0], x_range[1])
    return offspring

# Оптимизация генетического алгоритма
class GeneticAlgorithmOptimizer:
    def __init__(self, param_ranges, population_size, offspring_ratio, mutation_rate, mutation_strength, mutation_percent_genes,
                 mutation_by_replacement, elite_fraction, initialization_method, selection_percentage, early_stopping_patience, generations=400):
        self.param_ranges = param_ranges
        self.population_size = population_size
        self.offspring_ratio = offspring_ratio
        self.mutation_rate = mutation_rate
        self.mutation_strength = mutation_strength
        self.mutation_percent_genes = mutation_percent_genes
        self.mutation_by_replacement = mutation_by_replacement
        self.elite_fraction = elite_fraction
        self.initialization_method = initialization_method
        self.selection_percentage = selection_percentage
        self.early_stopping_patience = early_stopping_patience
        self.generations = generations
    
    def optimize(self):
        population = initialize_population(self.population_size, (-500, 500), 10, method=self.initialization_method)
        fitness_values = np.array([cached_objective_function(ind) for ind in population])

        best_solution = None
        best_fitness = float('inf')
        no_improvement_count = 0

        global_start_time = time.time()

        for generation in range(self.generations):
            selected_population, selected_fitness = select_top_individuals_with_elitism(
                population, fitness_values, self.selection_percentage, elite_count=int(self.elite_fraction * self.population_size)
            )
            num_offspring = int(len(selected_population) * self.offspring_ratio)
            offspring = generate_offspring(selected_population, num_offspring)
            offspring = mutate(offspring, self.mutation_rate, self.mutation_strength, self.mutation_percent_genes,
                               self.mutation_by_replacement, (-500, 500))

            combined_population = np.concatenate([selected_population, offspring])
            combined_fitness = np.array([cached_objective_function(ind) for ind in combined_population])

            final_population, final_fitness = select_top_individuals_with_elitism(
                combined_population, combined_fitness, self.population_size / len(combined_population),
                elite_count=int(self.elite_fraction * self.population_size)
            )

            population = final_population
            fitness_values = final_fitness

            current_best_fitness = np.min(final_fitness)
            current_best_solution = final_population[np.argmin(final_fitness)]

            if current_best_fitness < best_fitness:
                best_fitness = current_best_fitness
                best_solution = current_best_solution
                no_improvement_count = 0
            else:
                no_improvement_count += 1

            if no_improvement_count >= self.early_stopping_patience:
                # print(f"Early stopping на {generation+1} итерации.")
                break

        elapsed_time = time.time() - global_start_time
        return best_solution, best_fitness, generation + 1, elapsed_time

# Генерация случайных гиперпараметров
def generate_random_hyperparameters():
    hyperparameters = {
        "population_size": random.randint(100, 500),
        "offspring_ratio": random.uniform(1.5, 3.0),
        "mutation_rate": random.uniform(0.1, 0.5),
        "mutation_strength": random.uniform(0.1, 0.5),
        "mutation_percent_genes": random.uniform(0.1, 0.2),
        "mutation_by_replacement": random.choice([True]),
        "elite_fraction": random.uniform(0.05, 0.2),
        "initialization_method": random.choice(['random', 'lhs'])
    }
    return hyperparameters

# Количество внешних и внутренних прогонов
num_outer_runs = 10
num_inner_runs = 30
summary_results = []
header_saved = False
csv_filename = 'cust_shvefel_upd.csv'

# Внешний цикл для различных гиперпараметров
for outer_run in range(num_outer_runs):
    print(f"\n=== Внешний прогон {outer_run + 1} ===\n")
    
    hyperparams = generate_random_hyperparameters()
    
    results = []

    # Внутренний цикл для одного набора гиперпараметров
    for inner_run in range(num_inner_runs):
        optimizer = GeneticAlgorithmOptimizer(
            param_ranges={
                "x0": (-500, 500),
                "x1": (-500, 500),
                "x2": (-500, 500),
                "x3": (-500, 500),
                "x4": (-500, 500),
                "x5": (-500, 500),
                "x6": (-500, 500),
                "x7": (-500, 500),
                "x8": (-500, 500),
                "x9": (-500, 500),
            },
            **hyperparams,
            generations=400,
            selection_percentage=0.5,
            early_stopping_patience=30
        )

        best_solution, best_fitness, best_generation, best_generation_time = optimizer.optimize()

        run_data = {
            'Значение функции Швефеля': best_fitness,
            'Лучшая итерация': best_generation,
            'Время до лучшей итерации': best_generation_time
        }
        results.append(run_data)

    df_results = pd.DataFrame(results)
    median_fitness = df_results['Значение функции Швефеля'].median()
    mode_iteration = df_results['Лучшая итерация'].mode()[0]

    summary_row = {
        'Медиана значения функции Швефеля': median_fitness,
        'Мода лучшей итерации': mode_iteration,
        'Медиана времени до лучшей итерации': df_results['Время до лучшей итерации'].median()
    }

    for param, value in hyperparams.items():
        summary_row[param] = value

    # Добавляем строку в csv-файл
    df_summary_row = pd.DataFrame([summary_row])
    df_summary_row.to_csv(csv_filename, mode='a', header=not header_saved, index=False, float_format='%.6f')
    header_saved = True  # Чтобы сохранить заголовок только один раз

    # Вывод строки перед записью в файл (или после)
    print(f"Результаты для внешнего прогона {outer_run + 1}:")
    print(df_summary_row.to_string(index=False))

    # Очистка кэша и ручное удаление крупных объектов
    cached_objective_function.clear_cache()
    del df_results, df_summary_row, results
    gc.collect()

# Итоговая таблица не создается, поскольку все записывается в файл
print(f"\nВсе результаты сохранены в файл '{csv_filename}'.")



=== Внешний прогон 1 ===

Результаты для внешнего прогона 1:
 Медиана значения функции Швефеля  Мода лучшей итерации  Медиана времени до лучшей итерации  population_size  offspring_ratio  mutation_rate  mutation_strength  mutation_percent_genes  mutation_by_replacement  elite_fraction initialization_method
                         0.173398                   400                             2.14051              100         1.870012       0.455752           0.197684                0.151147                     True        0.051635                   lhs

=== Внешний прогон 2 ===

Результаты для внешнего прогона 2:
 Медиана значения функции Швефеля  Мода лучшей итерации  Медиана времени до лучшей итерации  population_size  offspring_ratio  mutation_rate  mutation_strength  mutation_percent_genes  mutation_by_replacement  elite_fraction initialization_method
                         0.148847                   400                            3.022013              142         2.268315       0.2